In [ ]:
$confirmedString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv).Content
$recoveredString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv).Content
$mortalityString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv).Content

In [ ]:
# Filter to the United States
$usConfirmedByArea = $confirmedString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US'}
$usRecoveredByArea  = $recoveredString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US'}
$usMortalityByArea  = $mortalityString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US'}

Write-Verbose "$($usConfirmedByArea.Count) confirmed cases Areas" -verbose
Write-Verbose "$($usRecoveredByArea.Count) recovery Areas" -verbose
Write-Verbose "$($usMortalityByArea.Count) mortality Areas" -verbose

In [ ]:
# Get unique dates on or after 2/15
$dates = $usConfirmedByArea[0] | Get-member -MemberType NoteProperty  | Where-Object {$_.Name -match '\d+/\d+/\d+' -and ([datetime] $_.Name) -ge ([datetime] '2/15/2020')} | Select-Object -ExpandProperty Name

In [ ]:
# Summarize the data for each date
$usSummary = @()
foreach($date in $dates)
{
    #Write-Verbose $date -Verbose
    $confirmedSum = ($usConfirmedByArea | Measure-Object -Sum -Property $date).Sum
    $recoveredSum = ($usRecoveredByArea | Measure-Object -Sum -Property $date).Sum
    $mortalitySum = ($usMortalityByArea | Measure-Object -Sum -Property $date).Sum
    $usSummary += [pscustomobject] @{
        date = [datetime] $date
        confirmed = $confirmedSum
        recovered = $recoveredSum
        deaths = $mortalitySum
        activeCases = $confirmedSum - ($mortalitySum + $recoveredSum)
    }
}
$usSummary = $usSummary | Sort-Object -Property Date
Write-Verbose "$($usSummary.Count) days" -verbose

In [ ]:
# Define the x-axis for the plot
$xaxis = [XPlot.Plotly.Graph+Xaxis]::new()
$xaxis.nticks = 23
# https://github.com/d3/d3-time-format/blob/master/README.md
$xaxis.tickformat = '%m-%d'

# Define the y-axis for the plot
$yaxis = [XPlot.Plotly.Graph+Yaxis]::new()

# Define the marker for a point
$marker = [XPlot.Plotly.Graph+Marker]::new()
$marker.color = "rgb(255, 0, 0)"
$marker.Size =15

# Define the marker for a point
$recoveredMarker = [XPlot.Plotly.Graph+Marker]::new()
$recoveredMarker.color = "rgb(0, 255, 0)"
$recoveredMarker.Size =10

# Define the marker for a point
$mortalityMarker = [XPlot.Plotly.Graph+Marker]::new()
$mortalityMarker.color = "rgb(0, 0, 0)"
$mortalityMarker.Size =10

# Define the marker for a point
$activeMarker = [XPlot.Plotly.Graph+Marker]::new()
$activeMarker.color = "rgb(255, 165, 0)"
$activeMarker.Size =5


In [ ]:
#Format the traces

$usConfirmedTrace = [XPlot.Plotly.Graph+Scatter]::new()

$usConfirmedTrace.x  = $usSummary.Date
$usConfirmedTrace.y  = $usSummary.Confirmed
$usConfirmedTrace.text = 'US Confirmed'
$usConfirmedTrace.marker  = $marker
$usConfirmedTrace.name = 'US Confirmed'

$usRecoveredTrace = [XPlot.Plotly.Graph+Scatter]::new()

$usRecoveredTrace.x  = $usSummary.Date
$usRecoveredTrace.y  = $usSummary.Recovered
$usRecoveredTrace.text = 'US Recovered'
$usRecoveredTrace.marker  = $recoveredMarker
$usRecoveredTrace.name = 'US Recovered'

$usMortalityTrace = [XPlot.Plotly.Graph+Scatter]::new()

$usMortalityTrace.x  = $usSummary.Date
$usMortalityTrace.y  = $usSummary.Deaths
$usMortalityTrace.text = 'US Deaths'
$usMortalityTrace.marker  = $mortalityMarker
$usMortalityTrace.name = 'US Deaths'

$usActiveTrace = [XPlot.Plotly.Graph+Scatter]::new()

$usActiveTrace.x  = $usSummary.Date
$usActiveTrace.y  = $usSummary.ActiveCases
$usActiveTrace.marker  = $activeMarker
$usActiveTrace.name = 'US Active Cases'


In [ ]:
# Create the chart
$layout = [XPlot.Plotly.Layout+Layout]::new()
$layout.width = 1024
$layout.height = 400

$layout.xaxis = $xaxis
$layout.yaxis = $yaxis
$layout.title = "John Hopkins - US - Coronavirus Data"
$layout.showlegend = $true
# Render the chart
New-PlotlyChart -trace $usConfirmedTrace,$usRecoveredTrace,$usMortalityTrace, $usActiveTrace -Layout $layout  | Out-Display